In [ ]:
import BioSimSpace as BSS
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
# Plot and save heatmaps of overlap matrixes

def getHeatMap(bound, free, title, reverse):
    f = ''
    if reverse: f = '_reverse'
    fig, axs = plt.subplots(ncols = 2, nrows = 1, figsize = (20, 10), constrained_layout = True)
    fig.supxlabel('λ index',fontsize = 14)
    fig.supylabel('λ index',fontsize = 14)
    
    cmap = mpl.cm.viridis
    bounds = [0, 0.1, 0.2, 0.4, 0.6, 0.8]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    fig.colorbar(mpl.cm.ScalarMappable(norm = norm, cmap = cmap))
    
    for col in range(2):
        if col == 0:
            leg = "Bound leg"
            matrix = np.array(bound)
        else:
            leg = "Free leg"
            matrix = np.array(free)
        lamb = matrix.shape[0]

        im = axs[col].imshow(matrix, norm = norm, cmap = cmap)   
        
        axs[col].set_xticks(np.arange(lamb), labels = np.arange(lamb))
        axs[col].set_yticks(np.arange(lamb), labels = np.arange(lamb))
        
        axs[col].set_title(leg, fontsize=16)
        axs[col].spines[:].set_visible(False)
        axs[col].grid(visible = True, which = "minor", axis = 'both', color = "w", linestyle = '-', linewidth = 3)
        
        for i in range(lamb):
            for j in range(lamb):
                text = axs[col].text(j, i, '{:.2f}'.format(matrix[i, j]), ha = "center", va = "center", color = "w", fontsize = 12)
    os.path.join('output'+f, title)
    fig.savefig(fname = os.path.join('output'+f, title))

In [ ]:
def getPertSys(testset, ref_ligand, protein, reverse = False):
    for index, row in testset.iterrows():
        title = row['Molecule Name']
        f = ''
        if reverse: f = '_reverse'
        path = os.path.join('pertSystems'+f, title)
        
        if os.path.exists(path):
            print("Perturtable system for " + title + " exists")
        else:
            # Parametrise the ligand 
            if os.path.exists(os.path.join('lig_par',title)):
                print("Parameterised ligand exists")
                ligand = BSS.IO.readMolecules([os.path.join('lig_par',title,'leap') +'.crd', os.path.join('lig_par',title,'leap') +'.top'])[0]
            else:
                print("Parameterising " + title +  " ...")
                ligand = BSS.IO.readMolecules(os.path.join('input','ligands_protonated',title) + '.pdb')[0]    
                ligand = BSS.Parameters.gaff2(ligand, work_dir = os.path.join('lig_par',title)).getMolecule()

            # Morph and combine systems
            if reverse:
                mapping = BSS.Align.matchAtoms(ref_ligand, ligand)
                ref_ligand = BSS.Align.rmsdAlign(ref_ligand, ligand, mapping)
                merged = BSS.Align.merge(ref_ligand, ligand, mapping)
            else:
                mapping = BSS.Align.matchAtoms(ligand, ref_ligand)
                ligand = BSS.Align.rmsdAlign(ligand, ref_ligand, mapping)
                merged = BSS.Align.merge(ligand, ref_ligand, mapping)
            complx = merged + protein 

            print("Solvating " + title + " ...")
            # Solvate the protein ligand complex and merged ligand in water
            complx_sol = BSS.Solvent.tip3p(molecule = complx, box = 3*[80*BSS.Units.Length.angstrom])
            merged_sol = BSS.Solvent.tip3p(molecule = merged, box = 3*[30*BSS.Units.Length.angstrom])
            
            print("Minimising " + title + " ...")
            minimisation_complex_path = os.path.join('minimisation'+f,title,'complex')
            minimisation_merged_path = os.path.join('minimisation'+f,title,'merged')
            # Create the minimisation protocol
            prot_minimisation = BSS.Protocol.Minimisation(steps = 5000)  
            # Minimise both legs
            proc_minimisation_complx = BSS.MD.run(complx_sol, 
                                                  prot_minimisation, 
                                                  work_dir = minimisation_complex_path,
                                                  gpu_support = True)   
            complx_minimised = proc_minimisation_complx.getSystem(block = True) 
            proc_minimisation_merged = BSS.MD.run(merged_sol, 
                                                  prot_minimisation, 
                                                  work_dir = minimisation_merged_path,
                                                  gpu_support = True)
            merged_minimised = proc_minimisation_merged.getSystem(block = True)

            print("Equilibrating " + title + " ...")
            equilibration_complex_path = os.path.join('equilibration'+f,title,'complex')
            equilibration_merged_path = os.path.join('equilibration'+f,title,'merged')            
            # Create the equilibration protocol
            prot_equilibration = BSS.Protocol.Equilibration(timestep = 2*BSS.Units.Time.femtosecond,
                                                            runtime = 400*BSS.Units.Time.picosecond)            
            # Equilibrate both legs
            proc_equilibration_complx = BSS.Process.Somd(complx_minimised, 
                                                         prot_equilibration,
                                                         work_dir = equilibration_complex_path,
                                                         platform = 'CUDA')
            proc_equilibration_complx.start()
            proc_equilibration_complx.wait()
            complx_equilibrated = proc_equilibration_complx.getSystem() 
            BSS.IO.savePerturbableSystem(os.path.join(path, title + '_bound'), complx_equilibrated) 
            print("Perturbable systems for " + title + " complex saved") 
            proc_equilibration_merged = BSS.Process.Somd(merged_minimised, 
                                                        prot_equilibration,
                                                        work_dir = equilibration_merged_path,
                                                        platform = 'CUDA')
            proc_equilibration_merged.start()
            proc_equilibration_merged.wait()
            merged_equilibrated = proc_equilibration_merged.getSystem()             
            BSS.IO.savePerturbableSystem(os.path.join(path, title + '_free'), merged_equilibrated)
            print("Perturbable systems for " + title + " merged saved")

In [ ]:
def getFreeEnergy(title, lam_bound, lam_free, reverse = False):
    # Read perturbable system
    f = ''
    if reverse: f = '_reverse'
    path = os.path.join('pertSystems'+f, title)
    boundSystem = BSS.IO.readPerturbableSystem(os.path.join(path,title+'_bound0.prm7'), os.path.join(path,title+'_bound0.rst7'),
                                               os.path.join(path,title+'_bound1.prm7'), os.path.join(path,title+'_bound1.rst7'))
    freeSystem = BSS.IO.readPerturbableSystem(os.path.join(path,title+'_free0.prm7'), os.path.join(path,title+'_free0.rst7'),
                                              os.path.join(path,title+'_free1.prm7'), os.path.join(path,title+'_free1.rst7'))
    
    # Create the free energy protocols
    prot_bound = BSS.Protocol.FreeEnergy(timestep = 2*BSS.Units.Time.femtosecond,
                                         runtime = 200*BSS.Units.Time.picosecond,
                                         num_lam = lam_bound)
    prot_free = BSS.Protocol.FreeEnergy(timestep = 2*BSS.Units.Time.femtosecond,
                                        runtime = 200*BSS.Units.Time.picosecond,
                                        num_lam = lam_free)

    # Initialise the free energy object for each leg
    fep_bound = BSS.FreeEnergy.Relative(boundSystem,
                                        prot_bound,
                                        engine = "somd",
                                        work_dir = os.path.join('somd'+f, title, 'bound'))
    fep_free = BSS.FreeEnergy.Relative(freeSystem,
                                       prot_free,
                                       engine = "somd",
                                       work_dir = os.path.join('somd'+f, title, 'free'))
    
    # Run FEP
    print("Computing FEP for " + title + " bound leg ...")
    fep_bound.run()
    fep_bound.wait()
    print("Computing FEP for " + title + " free leg ...")
    fep_free.run()
    fep_free.wait()
    
    # Get the PMF and the overlap matrix
    pmf_bound, overlap_bound = fep_bound.analyse()
    pmf_free, overlap_free = fep_free.analyse()
    
    # Compute the relative free-energy difference with PMF
    # dG_bound-dG_free = dG_ref,bind - dG_lig,bind
    binding_free_energy = BSS.FreeEnergy.Relative.difference(pmf_bound, pmf_free)
    
    return binding_free_energy, overlap_bound, overlap_free

In [ ]:
def savePertSys(testset, ref_id, protein_id, reverse = False):
    # Parameterise the reference ligand
    if os.path.exists("./" + ref_id):
        print("Parameterised reference ligand exists")
        ref_ligand = BSS.IO.readMolecules([os.path.join(ref_id, ref_id) + '.prm7', os.path.join(ref_id, ref_id) + '.rst7'])[0]
    else:
        print("Parameterising the reference ligand ...")
        ref_ligand = BSS.Parameters.gaff2(BSS.IO.readMolecules(os.path.join('input', 'ligands_protonated', ref_id) + '.pdb')[0], work_dir = "./reflig_par").getMolecule()
        BSS.IO.saveMolecules(os.path.join(ref_id, ref_id), ref_ligand, ["PRM7", "RST7"])

    # Add missing hydrogens and parameterise the protein
    if os.path.exists("./protein"):
        print("Parameterised protein exists")
        protein = BSS.IO.readMolecules(['./protein/protein.prm7', './protein/protein.rst7'])[0]
    else:
        print("Parameterising the protein ...")
        try:
            protein = BSS.Parameters.ff14SB(BSS.IO.readMolecules(os.path.join('input', protein_id) + '.pdb')[0], work_dir = "./protein_par_0").getMolecule()
        except:
            protein = BSS.Parameters.ff14SB(BSS.IO.readMolecules(['./protein_par_0/leap.crd','./protein_par_0/leap.top'])[0], work_dir = "./protein_par_1").getMolecule()
            BSS.IO.saveMolecules("./protein/protein", protein, ["PRM7", "RST7"])
            BSS.IO.saveMolecules("./protein/protein", protein, 'PDB')
    
    getPertSys(testset, ref_ligand, protein, reverse)

In [ ]:
# Run FEPs and save results

if __name__ == '__main__':
    ref_id = 'CVD-0007756'
    protein_id = '6LU7'
    reverse = True
    testset = pd.read_csv("./input/testset.csv")[1:]
    # run backward simulation for ligands CVD-0015217, CVD-0015032, CVD-0015218 id:2,6,10
    # testset = pd.read_csv("./input/testset.csv").loc[[2,6,10]]
    f = ''
    if reverse: f = '_reverse'
    out_dir = os.path.join('output'+f) 
    lam_bound = 15
    lam_free = 15  
    
    savePertSys(testset, ref_id, protein_id, reverse)
    
    if not os.path.exists(out_dir): os.makedirs(out_dir)
    with open(out_dir + '/result.txt', 'a') as f:
        f.writelines('Reference ligand ' + ref_id + '\n')

    for index, row in testset.iterrows():
        title = row['Molecule Name']
        FreeEnergy, overlap_bound, overlap_free = getFreeEnergy(title, lam_bound, lam_free, reverse)
        getHeatMap(overlap_bound, overlap_free, title, reverse)
        with open(out_dir + '/result.txt', 'a') as f:
            f.writelines(title + ',' + str(FreeEnergy[0]) + ',' + str(FreeEnergy[1]) + '\n')

In [ ]:
bound_1 = BSS.IO.readMolecules(['pertSystems_reverse/CVD-0015217/CVD-0015217_bound1.prm7','pertSystems_reverse/CVD-0015217/CVD-0015217_bound1.rst7'])
BSS.IO.saveMolecules("CVD-0015217_bound1", bound_1, 'PDB')

In [ ]:
bound_0 = BSS.IO.readMolecules(['pertSystems/CVD-0015032/CVD-0015032_bound1.prm7','pertSystems/CVD-0015032/CVD-0015032_bound1.rst7'])
BSS.IO.saveMolecules("CVD-0015232_bound0", bound_0, 'PDB')

In [ ]:
!jupyter nbconvert --to script BindingFE.ipynb